In [ ]:
import os
import re
import json
import fitz
import spacy
import faiss
import pickle
import win32com.client
from docx import Document
from RAGLibrary import Widgets, SegmentChunks

os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [ ]:
exceptions = {
    "a", "an", "the",
    "and", "but", "or", "nor", "for", "so", "yet",
    "at", "by", "in", "of", "on", "to", "from", "with", "as",
    "into", "like", "over", "under", "up", "down", "out", "upon", "onto",  
    "amid", "among", "between", "before", "after", "against"  
}

In [ ]:
widgets_list = Widgets.create_name_form()

In [ ]:
""" DEFINE """

""" GET PARENT VALUE """
data   = widgets_list[0] #HBox 1
keys   = widgets_list[1] #HBox 2
choose = widgets_list[2] #HBox 3

embedd_model = widgets_list[3]
search_egine = widgets_list[4]
rerank_model = widgets_list[5]
respon_model = widgets_list[6]
API_drop     = widgets_list[7]

# Hbox 4
chunk_input = widgets_list[8]

# HBox 5
level_value = widgets_list[9]

# Button
button_box  = widgets_list[10]


""" GET CHILDREN VALUE """

# HBox 1
file_name = data.children[0]
file_type = data.children[1]
path_end  = data.children[2]

# HBox 2
data_key = keys.children[0]
embe_key = keys.children[1]

# HBox 3
switch_model = choose.children[0]
merge_otp    = choose.children[1]

# HBox 4
level_input = chunk_input.children[0]
word_limit  = chunk_input.children[1]

# HBox 5
level_values = [child.value for child in level_value.children]

""" DEF VALUE """

# Get value
data_folder   = file_name.value
file_type_val = file_type.value

data_key_val  = data_key.value
embe_key_val  = embe_key.value

API_key_val = API_drop.value
switch      = switch_model.value
merge       = merge_otp.value
path_end_v  = path_end.value

embedding_model = embedd_model.value
searching_egine = search_egine.value
reranking_model = rerank_model.value
responing_model = respon_model.value

level_input_val = level_input.value
word_limit_val  = word_limit.value


""" DETAIL """

# Define
dcmt_path = f"../Doc/{data_folder}{path_end_v}"

base_folder = "../Data"
base_path = f"{base_folder}/{data_folder}/{file_type_val}_{data_folder}"

chunks_base    = f"{base_path}_Chunks.json"
json_file_path = f"{base_path}_Database.json"
schema_ex_path = f"{base_path}_Schema.json"
embedding_path = f"{base_path}_Embeds_{merge}"

torch_path  = f"{embedding_path}.pt"
faiss_path  = f"{embedding_path}.faiss"
mapping_path = f"{embedding_path}_mapping.json"
mapping_data = f"{embedding_path}_map_data.json"

FILE_TYPE    = file_type_val
DATA_KEY     = data_key_val
EMBE_KEY     = embe_key_val
SWITCH       = switch
EMBEDD_MODEL = embedding_model
SEARCH_EGINE = searching_egine
RERANK_MODEL = reranking_model
RESPON_MODEL = responing_model

if FILE_TYPE == "Data":
    MERGE = merge
else: 
    MERGE = "no_Merge"

API_KEY = API_key_val

SEARCH_ENGINE = faiss.IndexFlatIP

print("\n")
print(f"Embedder: {EMBEDD_MODEL}")
print(f"Searcher: {SEARCH_EGINE}")
print(f"Reranker: {RERANK_MODEL}")
print(f"Responer: {RESPON_MODEL}")
print(f"Data Key: {DATA_KEY}")
print(f"Embe Key: {EMBE_KEY}")
print(f"Dcment  : {dcmt_path}")
print(f"Chunked : {chunks_base}")
print(f"Database: {json_file_path}")
print(f"Torch   : {torch_path}")
print(f"Faiss   : {faiss_path}")
print(f"Mapping : {mapping_path}")
print(f"Map Data: {mapping_data}")
print(f"Schema  : {schema_ex_path}")
print(f"Model   : {SWITCH}")
print(f"Merge   : {MERGE}")
print(f"API Key : {API_KEY}")
print(f"Word    : {word_limit_val}")
print(f"Level   : {level_input_val}")
print(f"Level Values: {level_values}")

BASE


In [ ]:
# FORMAT TEXT
def format_text(text, case="upper"):
    if case == "upper":
        return text.upper()
    elif case == "Chapter":
        return ' '.join(word.capitalize() if word.lower() not in exceptions else word.lower() for word in text.split())
    return text

In [ ]:
# SIZE OF Chapter AND Title
def find_sizes(text_data):
    Chapter_size, sub_size = 0, 0
    
    for entry in text_data:
        text, font_size = entry["text"], entry["font_size"]
        if text.isupper():
            Chapter_size = max(Chapter_size, font_size)
        else:
            sub_size = max(sub_size, font_size)
    return Chapter_size, sub_size

In [ ]:
# ADD CHUNKS
def add_chunk(chunks, content):
    if content["Content"]:
        chunks.append(content.copy())
        content["Content"] = []

In [ ]:
# MAIN FUNCTION
def main(text_data, Chapter_size, sub_size):
    chunks = []
    content = {"Chapter": None, "Title": None, "Article": None, "Content": []}

    for entry in text_data:
        text, font_size = entry["text"], entry["font_size"]
        if not text:
            continue
        if text.isupper():
            if font_size == Chapter_size:
                add_chunk(chunks, content)
                content["Chapter"] = format_text(text, "upper")
                content["Title"] = None
                content["Article"] = None
            else:
                add_chunk(chunks, content)
                content["Article"] = format_text(text, "upper")
        else:
            if font_size == sub_size:
                add_chunk(chunks, content)
                content["Title"] = format_text(text, "Chapter")
                content["Article"] = None
            else:
                if content["Content"]:
                    last_sentence = content["Content"][-1]
                    if not last_sentence.endswith((".", "!", "?")):
                        content["Content"][-1] += " " + text
                    else:
                        content["Content"].append(text)
                else:
                    content["Content"].append(text)
    
    add_chunk(chunks, content)
    return chunks

In [ ]:
# CHUNKS BASE
text_data = ExtractData.extracted(dcmt_path)
Chapter_size, sub_size = find_sizes(text_data)
chunks = main(text_data, Chapter_size, sub_size)

In [ ]:
# EXPORT BASE
with open(chunks_base, "w", encoding="utf-8") as f:
    json.dump(chunks, f, indent=4, ensure_ascii=False)

print(f"Base data saved to {chunks_base}")

FINAL


In [ ]:
nlp = spacy.load("en_core_web_sm")
WORD_LIMIT = int(word_input.value) if word_input.value.isdigit() else 200

In [ ]:
SegmentChunks.process_json(chunks_base, json_file_path, Contents, word_limit, markers, nlp)